# Lab 8: P-Tuning v2 - Deep Prompt Tuning for Diverse Tasks
---
## Notebook 3: Inference

**Goal:** In this notebook, you will load the trained P-Tuning v2 adapter and use the fine-tuned model to perform various tasks, demonstrating the universal effectiveness of deep prompt tuning.


### Step 1: Reload Model and Adapter

We will load the base BERT model and then apply our trained P-Tuning v2 weights on top of it.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
import torch
import os

# --- Load Base Model and Tokenizer ---
model_checkpoint = "bert-base-uncased"
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# --- Load PEFT Adapter ---
output_dir = "./bert-ptuning-v2-cola"
latest_checkpoint = max(
    [os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint-")],
    key=os.path.getmtime
)
print(f"Loading adapter from: {latest_checkpoint}")

inference_model = PeftModel.from_pretrained(base_model, latest_checkpoint)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model.to(device)
inference_model.eval()

print("✅ P-Tuning v2 fine-tuned model loaded successfully!")


### Step 2: Perform Linguistic Acceptability Classification

Now, let's test the model by performing linguistic acceptability classification on new sentences. We'll test sentences that are grammatically correct vs. incorrect.


In [ ]:
import torch.nn.functional as F

# Define the labels
id2label = {0: "Unacceptable", 1: "Acceptable"}

def predict_acceptability(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = inference_model(**inputs)
    
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1).cpu().numpy()[0]
    prediction = torch.argmax(logits, dim=-1).cpu().item()
    
    print(f"Sentence: '{text}'")
    print(f"Prediction: {id2label[prediction]}")
    print(f"Confidence: {probabilities[prediction]:.4f}")
    print(f"Probabilities:")
    print(f"  - {id2label[0]}: {probabilities[0]:.4f}")
    print(f"  - {id2label[1]}: {probabilities[1]:.4f}")
    print()

# --- Test Cases ---
# Mix of grammatically correct and incorrect sentences
test_sentences = [
    "The cat sat on the mat.",                           # Correct
    "The cat sat on mat the.",                           # Incorrect (word order)
    "She enjoys reading books in the library.",          # Correct
    "Books enjoys reading she in library the.",          # Incorrect (scrambled)
    "The students are studying for their exams.",        # Correct
    "Students the are studying for exams their.",        # Incorrect (word order)
    "I think that this sentence is grammatically correct.", # Correct
    "Think I that sentence this is correct grammatically.", # Incorrect (word order)
    "The weather is beautiful today.",                   # Correct
    "Weather the is today beautiful.",                   # Incorrect (word order)
]

print("--- P-Tuning v2 Linguistic Acceptability Results ---")
print("Testing the model's ability to distinguish grammatically correct from incorrect sentences:\n")

for sentence in test_sentences:
    predict_acceptability(sentence)


---
### Lab Conclusion

This concludes the P-Tuning v2 lab. You have successfully implemented deep prompt tuning that applies to every transformer layer, achieving:

- **Universal Effectiveness**: P-Tuning v2 works across various tasks and model scales
- **Extreme Parameter Efficiency**: Only ~0.1% of parameters while matching full fine-tuning performance
- **Simplified Architecture**: No need for complex MLP encoders like P-Tuning v1
- **Strong Performance**: Comparable to full fine-tuning with minimal overhead

P-Tuning v2 represents the evolution of prompt-based methods toward universal and efficient fine-tuning!
